In [8]:
#LOADING THE PREPROCESSED DS FOR WORKING ON IT
import pickle

with open('/content/preprocessed_squad.pkl', 'rb') as f:
    loaded_df = pickle.load(f)

print("Preprocessed data loaded from preprocessed_squad.pkl")
# Checking few rows
print(loaded_df.head())


Preprocessed data loaded from preprocessed_squad.pkl
                         id                     title  \
0  5733be284776f41900661182  University_of_Notre_Dame   
1  5733be284776f4190066117f  University_of_Notre_Dame   
2  5733be284776f41900661180  University_of_Notre_Dame   
3  5733be284776f41900661181  University_of_Notre_Dame   
4  5733be284776f4190066117e  University_of_Notre_Dame   

                                             context  \
0  Architecturally, the school has a Catholic cha...   
1  Architecturally, the school has a Catholic cha...   
2  Architecturally, the school has a Catholic cha...   
3  Architecturally, the school has a Catholic cha...   
4  Architecturally, the school has a Catholic cha...   

                                            question  \
0  To whom did the Virgin Mary allegedly appear i...   
1  What is in front of the Notre Dame Main Building?   
2  The Basilica of the Sacred heart at Notre Dame...   
3                  What is the Grotto at No

In [6]:
!pip install transformers datasets torch nltk spacy

In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the fine-tuned T5 model and tokenizer from Hugging Face
model_name = "Drashtip/fine_tuned_t5_squad"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [23]:
# List files in the current directory
print(os.listdir('.'))


['.config', 'preprocessed_squad.pkl', 'sample_data']


In [32]:
#error
# Load the preprocessed SQuAD dataset
try:
    with open('preprocessed_squad.pkl', 'rb') as f:
        preprocessed_data = pickle.load(f)
    print("Data loaded successfully.")
    # Check the type and structure of the loaded data
    print(f"Data type: {type(preprocessed_data)}")
    print(f"First entry: {preprocessed_data[0]}")  # Display the first entry to verify the structure
except Exception as e:
    print(f"An error occurred while loading the file: {e}")


Data loaded successfully.
Data type: <class 'pandas.core.frame.DataFrame'>
An error occurred while loading the file: 0


In [33]:
# Display the first few rows and columns of the DataFrame to understand its structure
print(preprocessed_data.head())
print(preprocessed_data.columns)


                         id                     title  \
0  5733be284776f41900661182  University_of_Notre_Dame   
1  5733be284776f4190066117f  University_of_Notre_Dame   
2  5733be284776f41900661180  University_of_Notre_Dame   
3  5733be284776f41900661181  University_of_Notre_Dame   
4  5733be284776f4190066117e  University_of_Notre_Dame   

                                             context  \
0  Architecturally, the school has a Catholic cha...   
1  Architecturally, the school has a Catholic cha...   
2  Architecturally, the school has a Catholic cha...   
3  Architecturally, the school has a Catholic cha...   
4  Architecturally, the school has a Catholic cha...   

                                            question  \
0  To whom did the Virgin Mary allegedly appear i...   
1  What is in front of the Notre Dame Main Building?   
2  The Basilica of the Sacred heart at Notre Dame...   
3                  What is the Grotto at Notre Dame?   
4  What sits on top of the Main Building

In [34]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the fine-tuned T5 model and tokenizer from Hugging Face
model_name = "Drashtip/fine_tuned_t5_squad"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to generate a question from a given context
def generate_question(context):
    input_text = f"generate question: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate question
    output_ids = model.generate(inputs['input_ids'], max_length=50, num_beams=5, early_stopping=True)

    # Decode generated question
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question

# Use a sample context from the DataFrame to generate a question
sample_context = preprocessed_data['context'][0]  # Modify if 'context' is named differently
generated_question = generate_question(sample_context)

print(f"Context: {sample_context}")
print(f"Generated Question: {generated_question}")


Context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Generated Question: Where did the Virgin Mary appear to Saint Bernadette Soubirous?


In [35]:
# checking the scores for evaluation !
# starting with rouge and bleu.

In [36]:
#lib installation
!pip install rouge-score nltk

from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score

# Example reference question (ground truth) from your dataset
reference_question = "Where did the Virgin Mary appear to Saint Bernadette?"

# Generated question from the model
generated_question = "Where did the Virgin Mary appear to Saint Bernadette Soubirous?"

# Convert reference and generated questions to lists of tokens for BLEU
reference_tokens = [reference_question.split()]
generated_tokens = generated_question.split()

# 1. Calculate BLEU Score
bleu_score = sentence_bleu(reference_tokens, generated_tokens)
print(f"BLEU Score: {bleu_score:.4f}")

# 2. Calculate ROUGE Score (tokenize input for rouge_scorer)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(" ".join(reference_tokens[0]), " ".join(generated_tokens))
print(f"ROUGE Scores: {rouge_scores}")

BLEU Score: 0.7598
ROUGE Scores: {'rouge1': Score(precision=0.9, recall=1.0, fmeasure=0.9473684210526316), 'rougeL': Score(precision=0.9, recall=1.0, fmeasure=0.9473684210526316)}
